In [1]:
import pandas as pd
from numpy.random import choice
import my_pickle as mp
import matplotlib.pyplot as plt

In [2]:
user_df = mp.unpickle_it('data_user')
convo_df = mp.unpickle_it('data_convo')

print("Make sure we're not useing ECT data")
print(pd.to_datetime(convo_df.timestamp.max()*1000000))

Make sure we're not useing ECT data
2017-10-02 01:35:58.644000


In [3]:
user_df.head()

,created,updated,activeAt,available,about,has_about,len_about,birthday,age,gender,...,isStudent,petsOk,smokingOk,onboarded,messages_sent,responses_received,attractiveness,messages_received,responses_sent,responsiveness
uid,,,,,,,,,,,,,,,,,,,,,
07cSOmlL0F,2016-11-15 22:54:02.653,2017-02-08 00:06:05.667,NaT,NaT,,False,0,1990-11-16 21:00:00,28.0,male,...,NaN,1.0,0.0,0.0,1.0,0.0,0.000000,1.0,0.0,0.000000
0PZ0jSiV11,2016-11-16 03:36:00.307,2017-03-13 19:38:46.405,NaT,NaT,,False,0,1989-11-24 21:00:00,29.0,male,...,NaN,1.0,1.0,0.0,0.0,0.0,0.392391,3.0,0.0,0.000000
1TVL6H99Hl,2017-03-01 06:07:51.386,2017-08-02 16:34:25.313,NaT,NaT,,False,0,1990-05-02 21:00:00,28.0,female,...,NaN,1.0,0.0,0.0,0.0,0.0,0.392391,0.0,0.0,0.981288
25mzfm6r5U,2017-02-27 08:38:14.463,2017-02-27 08:38:16.704,NaT,NaT,,False,0,NaT,NaN,male,...,NaN,NaN,NaN,NaN,0.0,0.0,0.392391,0.0,0.0,0.981288
2IWYvNu4gB,2017-03-02 01:31:36.616,2017-03-31 17:33:34.831,NaT,NaT,,False,0,NaT,NaN,female,...,NaN,NaN,NaN,NaN,0.0,0.0,0.392391,0.0,0.0,0.981288


# FILES THAT SHOULD BE MOVED TO PY FILE

In [4]:
def get_rent_range(first_user, second_user):
    max1 = first_user.maxCost
    max2 = second_user.maxCost
    min1 = first_user.minCost
    min2 = second_user.minCost
    if max1*max2*min1*min2 > 0:
        upper = min(first_user.maxCost,second_user.maxCost)
        lower = max(first_user.minCost,second_user.minCost)
        if upper-lower > 0: return upper-lower
        else: return 0
    else: return 175
    
def test_get_rent_range():    
    yes1 = '0a9yOPKFSH'
    yes2 = '013LzOrVju'
    no1 = '0UBLgJIHgz'
    no2 = '0EWNOmyQmW'
    low = '01DE0NCjwh'
    high = '02GDyQPLII'
    print(get_rent_range(user_df.loc[yes1], user_df.loc[yes2])) # should be 150
    print(get_rent_range(user_df.loc[yes1], user_df.loc[no1]))  # should be 175
    print(get_rent_range(user_df.loc[no2], user_df.loc[no1]))   # should be 175
    print(get_rent_range(user_df.loc[low], user_df.loc[high]))  # should be 0
    
def get_average_overlap(df,samples):
    # to find average overlap of two users' rent ranges
    # comes out to about $175
    s = 0
    for n in range(samples):
        two_users = choice(user_df[user_df.cost_range>0].index,2)
        overlap = user_df.loc[two_users,'maxCost'].min()-user_df.loc[two_users,'minCost'].max()
        if overlap < 0: overlap = 0
        s += overlap
    return s/samples


def roommate_rules(roommates):
    if roommates > 4: return 3
    elif roommates > 1: return 2
    elif roommates > 0: return 1
    else: return 0
    
def get_similar_roommates(first_user, second_user):
    roommates1 = roommate_rules(first_user.numRoommates)
    roommates2 = roommate_rules(second_user.numRoommates)
    return abs(roommates1-roommates2)


def get_ammenities_overlap(first, second):
    if isinstance(first.amenities, float) or isinstance(second.amenities, float):
        return 0
    else:
        return len(set(first.amenities).intersection(set(second.amenities))) + 1

def test_ammenities_overlap():    
    user0 = user_df.index[0]   # [Dishwasher]
    user1 = user_df.index[1]   # NaN
    user2 = user_df.index[2]   # [Laundry, A/C, Dishwasher, Private Bath]
    user8 = user_df.index[8]   # [Laundry]

    a = '8Ws3QWmC4B'
    b = '52lmw7KQcq'
    
    get_ammenities_overlap(user_df.loc[str(a)], second = user_df.loc[str(b)])

def get_hobbies_overlap(first, second):
    if isinstance(first.hobbies, float) or isinstance(second.hobbies, float):
        return 0
    else:
        return len(set(first.hobbies).intersection(set(second.hobbies))) + 1

# FAILED METRICS
# def get_inverse_distance(first_user, second_user):
#     x1 = first_user.longitude
#     x2 = second_user.latitude
#     y1 = first_user.longitude
#     y2 = second_user.latitude
#     dist = ((x2-x1)**2+(y2-y1)**2)**(.5)
# #     if dist > 0: return dist
# #     else: return 1
#     return ((x2-x1)**2+(y2-y1)**2)**(-.5)

# def get_urgency(timestamp, second_user):
#     time_of_message = pd.to_datetime(timestamp*1000000)
#     time_of_looking
#     print(time_of_message)
# get_urgency(1479148499453, 'x')

In [10]:
def feature_time(df, user_df):
    '''
    age_dif: difference in ages between users
    rent_overlap: buy how much do their ideal rent ranges overlap
    same_gender: m/m or f/f
    same_relate:  are they both in relationships or single?
    same_clean: are the both clean/messy
    same_night: are they both early-birds or night owls?
    same_student: are they both students?
    sender_attractiveness: do people generally respond to this senders messages?
    receiver_selectivity: does this receiver generally respond to peopls messages?
    '''
    
    ad,ro,di,rn,ao,ho,sg,sr,sc,sn,ss,sm,st,sa,rr = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[]

    for index, row in df.iterrows():
        first = user_df.loc[str(row.first_uid)]
        second = user_df.loc[str(row.second_uid)]
        ad.append(abs(first.age - second.age))
        ro.append(get_rent_range(first, second))
#         di.append(get_inverse_distance(first, second))
        rn.append(get_similar_roommates(first, second))
        ao.append(get_ammenities_overlap(first, second))
        ho.append(get_hobbies_overlap(first, second))
        sg.append(first.gender == second.gender)
        sr.append(first.inRelationship == second.inRelationship)
        sc.append(first.isClean == second.isClean)
        sn.append(first.isNight == second.isNight)
        ss.append(first.isStudent == second.isStudent)
        sm.append(first.smokingOk == second.smokingOk)
        st.append(first.term == second.term)
        sa.append(first.attractiveness)
        rr.append(second.responsiveness)

    print(len(df))
    print(len(sr))
    df['age_dif'] = ad
    df['rent_overlap'] = ro
#     df['inverse_distance'] = di
    df['roommate_num_sim'] = rn
    df['amenities_overlap'] = ao
    df['hobbies_overlap'] = ho
    df['same_gender'] = sg
    df['same_relate'] = sr
    df['same_clean'] = sc
    df['same_night'] = sn
    df['same_student'] = ss
    df['same_smoking'] = sm
    df['same_term'] = st
    df['sender_attractiveness'] = sa
    df['receiver_responsiveness'] = rr
    
    TF = {True: 1, False: 0}
    col_to_TF = ['same_gender','same_relate','same_clean','same_night','same_student','same_smoking','same_term']
    for col in col_to_TF:
        df[col] = df[col].map(TF)
    
    return df

In [11]:
X = feature_time(convo_df, user_df)
X.head()

11396
11396


,conv_id,response,first_uid,second_uid,first_mid,second_mid,timestamp,age_dif,rent_overlap,roommate_num_sim,...,hobbies_overlap,same_gender,same_relate,same_clean,same_night,same_student,same_smoking,same_term,sender_attractiveness,receiver_responsiveness
16296,rKdn0zblFiRVDUEmDHyQ,True,rKdn0zblFi,RVDUEmDHyQ,-KWKghvlpF67UOv-iXXO,-KWKg_ybWV4qbG02NYrb,1478906535730,11.0,175.0,2,...,2,1,0,0,1,0,0,0,1.0,0.0
9241,aXnWmLjpaERVDUEmDHyQ,False,aXnWmLjpaE,RVDUEmDHyQ,-KWLsSEmEJgH3bhJupE7,None,1478926390334,11.0,175.0,1,...,1,0,1,0,1,0,0,0,0.0,0.0
1069,8Ws3QWmC4B52lmw7KQcq,True,8Ws3QWmC4B,52lmw7KQcq,-KWPb8hd_DbLpY1eIUlT,-KWQNe2ghZCGV0aywnKp,1478988962771,2.0,1000.0,0,...,3,1,1,1,1,0,1,0,0.5,0.0
9414,b1HYtkC11e92RR2gpmjr,False,b1HYtkC11e,92RR2gpmjr,-KWQMn5WmLIKbUOt9a0O,None,1479001715202,1.0,0.0,2,...,2,1,1,1,0,0,1,1,0.5,0.0
4142,KnmYDsn5Q492RR2gpmjr,False,KnmYDsn5Q4,92RR2gpmjr,-KWQNFIghvLurnI-8epo,None,1479001834795,1.0,175.0,1,...,2,0,1,0,0,0,1,0,0.0,0.0


# SAVE AS PICKLE FILE

In [12]:
mp.pickle_it(X,'data_features')